In [1]:
import os
import sys
sys.path.append("../..")
import phage_init

In [2]:
#load the saved matrices
import pickle
train_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"train_x_data2_full.p"), "rb" ) )
test_X_total=pickle.load(open( os.path.join(phage_init.data_dir,"test_x_data2_full.p"), "rb" ) )
train_Y=pickle.load(open( os.path.join(phage_init.data_dir,"train_y_data2_full.p"), "rb" ) )
test_Y=pickle.load(open( os.path.join(phage_init.data_dir,"test_y_data2_full.p"), "rb" ) )
mean_total=pickle.load(open( os.path.join(phage_init.data_dir,"mean_data2_full.p"), "rb" ) )
std_total=pickle.load(open( os.path.join(phage_init.data_dir,"std_data2_full.p"), "rb" ) )

In [3]:
di_train     = train_X_total[:,0:400]
tri_train    = train_X_total[:,400:8400]
di_sc_train  = train_X_total[:,8400:8449] 
tri_sc_train = train_X_total[:,8449:8792]
tt_train     = train_X_total[:,8792:]

In [4]:
di_test     = test_X_total[:,0:400]
tri_test    = test_X_total[:,400:8400]
di_sc_test  = test_X_total[:,8400:8449] 
tri_sc_test = test_X_total[:,8449:8792]
tt_test     = test_X_total[:,8792:]

In [5]:
mean_di     = mean_total[0:400]
mean_tri    = mean_total[400:8400]
mean_di_sc  = mean_total[8400:8449] 
mean_tri_sc = mean_total[8449:8792]
mean_tt     = mean_total[8792:]

In [6]:
std_di     = std_total[0:400]
std_tri    = std_total[400:8400]
std_di_sc  = std_total[8400:8449] 
std_tri_sc = std_total[8449:8792]
std_tt     = std_total[8792:]

In [7]:
#import keras and numpy
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [10]:
#print(di_train.shape)
print(tt_train.shape)
#train_X =numpy.concatenate((tri_sc_train,tt_train),axis=1)
#test_X  =numpy.concatenate((tri_sc_test,tt_test),axis=1)
#train_X=di_train
#test_X=di_test
train_X=train_X_total
test_X=test_X_total
model_name='all_data2.h5'
print(train_X.shape)
print(test_X.shape)

(50000, 8)
(50000, 8800)
(7774, 8800)


In [11]:
#this list the devices, just making sure there is a GPU present, you might be fine with no GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9622210027420150609
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 17249263681714498435
physical_device_desc: "device: XLA_CPU device"
]


In [12]:
print(train_X.shape[1])
f_num=train_X.shape[1]


8800


In [13]:
#model with 2 leyers of 100 LSTM neurons
model = Sequential()
opt=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.add(Dense(f_num, input_dim=f_num, kernel_initializer='random_uniform',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(200,activation='relu'))
#model.add(Dense(200,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8800)              77448800  
_________________________________________________________________
dropout_1 (Dropout)          (None, 8800)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 200)               1760200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               40200     
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                2010      
Total para

In [15]:
model.fit(train_X, train_Y, epochs=200,verbose=1, batch_size=2000)

Epoch 1/200
50000/50000 [==============================] - 59s 1ms/step - loss: 0.3803 - acc: 0.8976
Epoch 2/200
50000/50000 [==============================] - 56s 1ms/step - loss: 0.2771 - acc: 0.9204
Epoch 3/200
50000/50000 [==============================] - 56s 1ms/step - loss: 0.2083 - acc: 0.9382
Epoch 4/200
50000/50000 [==============================] - 56s 1ms/step - loss: 0.1660 - acc: 0.9495
Epoch 5/200
50000/50000 [==============================] - 57s 1ms/step - loss: 0.1393 - acc: 0.9560
Epoch 6/200
50000/50000 [==============================] - 56s 1ms/step - loss: 0.1178 - acc: 0.9618
Epoch 7/200
50000/50000 [==============================] - 61s 1ms/step - loss: 0.1016 - acc: 0.9660
Epoch 8/200
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0912 - acc: 0.9702
Epoch 9/200
50000/50000 [==============================] - 56s 1ms/step - loss: 0.0809 - acc: 0.9732
Epoch 10/200
50000/50000 [==============================] - 57s 1ms/step - loss: 0.0721 - a

50000/50000 [==============================] - 55s 1ms/step - loss: 0.0220 - acc: 0.9954
Epoch 161/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0226 - acc: 0.9950
Epoch 162/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0249 - acc: 0.9947
Epoch 163/200
50000/50000 [==============================] - 55s 1ms/step - loss: 0.0254 - acc: 0.9946
Epoch 164/200
50000/50000 [==============================] - 55s 1ms/step - loss: 0.0243 - acc: 0.9948
Epoch 165/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0237 - acc: 0.9946
Epoch 166/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0226 - acc: 0.9950
Epoch 167/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0222 - acc: 0.9951
Epoch 168/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.0220 - acc: 0.9949
Epoch 169/200
50000/50000 [==============================] - 54s 1ms/step - loss: 0.022

In [22]:
scores = model.evaluate(test_X, test_Y, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

7774/7774 [==============================] - 8s 1ms/step
Accuracy: 91.47%


In [23]:
#you can save the as a file so you dont have to train it every time
model.save( os.path.join(phage_init.model_dir,model_name))

In [ ]:
from sklearn.metrics import classification_report


test_Y_index = test_Y.argmax(axis=1) # Convert one-hot to index
test_Y_predicted = model.predict_classes(test_X)
labels_names=["Major capsid","Minor capsid","Baseplate","Major tail","Minor tail","Portal","Tail fiber",
             "Tail shaft","Collar","Head-Tail joining"]
print(classification_report(test_Y_index, test_Y_predicted, target_names=labels_names ))
#print (test_Y_index)

In [ ]:
from collections import Counter
zz=Counter(test_Y_index)
sample_w=[zz[i] for i in range(0,10,1)]
print(zz)
print(sample_w)
print()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

CM=confusion_matrix(test_Y_index, test_Y_predicted)
CM_n=CM/numpy.array(sample_w)[:,None]
scale_up=1.3
plt.figure(figsize=[6.4*scale_up, 4.8*scale_up])
plt.imshow(CM_n, interpolation='nearest')
plt.title('CM')
plt.colorbar()
tick_marks = numpy.arange(len(labels_names))
plt.xticks(tick_marks, labels_names, rotation=90)
plt.yticks(tick_marks, labels_names)
fmt = '.2f'
for i, j in itertools.product(range(CM_n.shape[0]), range(CM_n.shape[1])):
        plt.text(j, i, format(CM_n[i, j], fmt),horizontalalignment="center",verticalalignment='center',
                color="white" if CM_n[i, j] < 0.25 else "black")
plt.show()